In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from scipy.stats import norm


import inmoose
from inmoose.limma import *

#Find the utils Directory
sys.path.append(os.path.abspath("../../"))
from src.utils.ConvertTextToCsv import TextToCsv

sys.path.append(os.path.abspath("../../"))
from src.utils.Preprocessing import elimnation_zeros, total_type_len_type_cancer

%matplotlib inline

In [16]:
df_clincal_data = pd.read_csv("../../data/raw/brca_tcga_pub2015_clinical_data.tsv", sep='\t')
list_df = total_type_len_type_cancer(df_clincal_data)
df_clincal_data["Tumor-Cancer"] = list_df

Luminal A: 330 - Total(%): 0.40
Luminal B: 81 - Total(%):0.10
HER2-enriched: 23 - Total(%):0.03
TNBC: 85 - Total(%)0.10 
UNK: 298 - Total(%) 0.36


In [17]:
df_clincal_data["Tumor-Cancer"].unique()

array(['<UNK>', 'Luminal A', 'TNBC', 'Luminal B', 'HER2-enriched'],
      dtype=object)

In [18]:
df_mRNA = pd.read_csv("../../data/raw/data_mrna_seq_v2_rsem.txt", header=None)
df_mRNA_transformed = TextToCsv("../../data/raw/data_mrna_seq_v2_rsem.txt")

Shape of the CSV: (20440, 819)


In [19]:
df_mRNA_transformed = df_mRNA_transformed.drop(columns=["Hugo_Symbol", "Entrez_Gene_Id"], axis=0)
df_mrna = df_mRNA_transformed.T.reset_index()
df_mRNA_final = df_mrna.rename(columns={"index":"Sample ID"})

In [20]:
df_merged = pd.merge(df_mRNA_final, df_clincal_data, right_on="Sample ID", left_on="Sample ID")


In [46]:
comparation_df = df_merged.loc[
    df_merged["Tumor-Cancer"].isin(["Luminal A", "Luminal B", "TNBC", "HER2-enriched"]),
    ["Tumor-Cancer"] + list(df_merged.columns[1:20441])
]
comparation_df["Tumor-Cancer"].unique()

array(['Luminal A', 'TNBC', 'Luminal B', 'HER2-enriched'], dtype=object)

In [47]:
zeros_reduced_df = elimnation_zeros(comparation_df)

Max of zeros per row in the dataset: 519
Avg of zeros per row in the dataset: 74.00381604696673
Median of zeros per row in the dataset: 0.0
Min of zeros per row in the dataset: 0
After the 0 elimination: 16270


In [48]:
#zeros_reduced_df.iloc[:,1:] = zeros_reduced_df.iloc[:, 1:].astype(int)
print(f"Samples: {zeros_reduced_df.shape[0]}, Genes: {zeros_reduced_df.shape[1]}")

Samples: 519, Genes: 16270


In [49]:
metadata = pd.DataFrame(zeros_reduced_df["Tumor-Cancer"], index=zeros_reduced_df.index)
metadata.columns = ["Tumor_Cancer"]
counts_data = zeros_reduced_df.drop(columns=["Tumor-Cancer"])

counts_data = np.log2(counts_data + 1)
expr = counts_data.T
print(f"Genes {counts_data.shape[0]}, Samples {counts_data.shape[1]}")
print("metadata rows:", metadata.shape[0])
print("counts_data rows:", counts_data.shape[0])
print("counts_data cols:", counts_data.shape[1])
(expr.columns == metadata.index).all()

Genes 519, Samples 16269
metadata rows: 519
counts_data rows: 519
counts_data cols: 16269


np.True_

In [60]:
metadata_aligned = metadata.loc[expr.columns].copy()
dummies = pd.get_dummies(metadata_aligned["Tumor_Cancer"]).astype(float)
design = dummies
print(design.shape)
print(design.columns)

dummies

(519, 4)
Index(['HER2-enriched', 'Luminal A', 'Luminal B', 'TNBC'], dtype='object')


,HER2-enriched,Luminal A,Luminal B,TNBC
2,0.0,1.0,0.0,0.0
6,0.0,1.0,0.0,0.0
7,0.0,0.0,0.0,1.0
8,0.0,0.0,1.0,0.0
10,0.0,0.0,0.0,1.0
...,...,...,...,...
812,0.0,1.0,0.0,0.0
813,0.0,1.0,0.0,0.0
814,0.0,0.0,0.0,1.0
815,0.0,1.0,0.0,0.0


In [61]:
#This is for fitting the models
limma_fit_models = lmFit(obj=expr, design=design)